In [ ]:
# для colab
import os, sys, subprocess

REPO = "andersonTheCat/witch_practicum"
BRANCH = "main"
DEST = "/content/_repo"

if not os.path.exists(DEST):
    subprocess.run(
        ["git", "clone", "--depth", "1", "-b", BRANCH, f"https://github.com/{REPO}.git", DEST],
        check=True
    )
else:
    subprocess.run(["git", "-C", DEST, "pull", "--ff-only"], check=True)

if DEST not in sys.path:
    sys.path.insert(0, DEST)

print("(=^..^=) repo ready at", DEST)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

from meowmeow import Vec, Mat, display_latex

In [ ]:
def _err(msg):
    return ValueError("мяк, " + str(msg))

In [ ]:
def _p(x):
    return 1.0 / (2.0 + x)

In [ ]:
def _pprime(x):
    return -1.0 / ((2.0 + x) ** 2)

In [ ]:
def _r(x):
    return np.cos(x)

In [ ]:
def _f(x):
    return 1.0 + x

In [ ]:
def _canon_ref_kind(s):
    s = (s or "").strip().lower()
    if s in ("", "scipy", "solve_bvp"):
        return "scipy"
    if s in ("grid", "thin", "тонкая", "сетк", "сетка"):
        return "grid"
    return s

In [ ]:
def _thomas_solve(a, b, c, d):
    a = np.asarray(a, dtype=float)
    b = np.asarray(b, dtype=float)
    c = np.asarray(c, dtype=float)
    d = np.asarray(d, dtype=float)

    n = b.size
    if d.size != n:
        raise _err("thomas: RHS размер не совпал")
    if a.size != n - 1 or c.size != n - 1:
        raise _err("thomas: диагонали должны быть длины n-1")

    cp = np.zeros(n - 1, dtype=float)
    dp = np.zeros(n, dtype=float)

    den = b[0]
    if abs(den) < 1e-300:
        raise _err("thomas: нулевой элемент на главной диагонали (b[0])")
    cp[0] = c[0] / den
    dp[0] = d[0] / den

    for i in range(1, n - 1):
        den = b[i] - a[i - 1] * cp[i - 1]
        if abs(den) < 1e-300:
            raise _err(f"thomas: нулевой элемент диагонали на шаге i={i}")
        cp[i] = c[i] / den
        dp[i] = (d[i] - a[i - 1] * dp[i - 1]) / den

    den = b[n - 1] - a[n - 2] * cp[n - 2]
    if abs(den) < 1e-300:
        raise _err("thomas: деление на почти нулевой диагональный элемент")
    dp[n - 1] = (d[n - 1] - a[n - 2] * dp[n - 2]) / den

    x = np.zeros(n, dtype=float)
    x[n - 1] = dp[n - 1]
    for i in range(n - 2, -1, -1):
        x[i] = dp[i] - cp[i] * x[i + 1]
    return x

In [ ]:
def _assemble_main_tridiag_dirichlet(n, a=-1.0, b=1.0, ua=0.0, ub=0.0):
    h = (b - a) / n
    x = a + h * np.arange(n + 1)

    if n < 2:
        raise _err("n должен быть >= 2")

    m = n - 1
    lower = np.zeros(m - 1, dtype=float)
    diag = np.zeros(m, dtype=float)
    upper = np.zeros(m - 1, dtype=float)
    rhs = np.zeros(m, dtype=float)

    Ais = np.zeros(m, dtype=float)
    Bis = np.zeros(m, dtype=float)
    Cis = np.zeros(m, dtype=float)
    Gis = np.zeros(m, dtype=float)
    RHS = np.zeros(m, dtype=float)

    for i in range(1, n):
        xi = x[i]
        pL = _p(xi - 0.5 * h)
        pR = _p(xi + 0.5 * h)

        Ai = -pL / (h * h)
        Ci = -pR / (h * h)
        Bi = (pL + pR) / (h * h) + _r(xi)
        Gi = _f(xi)

        k = i - 1
        diag[k] = Bi
        rhs[k] = Gi

        if k - 1 >= 0:
            lower[k - 1] = Ai
        else:
            rhs[k] -= Ai * ua

        if k + 1 <= m - 1:
            if k < m - 1:
                upper[k] = Ci
        else:
            rhs[k] -= Ci * ub

        Ais[k] = Ai
        Bis[k] = Bi
        Cis[k] = Ci
        Gis[k] = Gi
        RHS[k] = rhs[k]

    return x, h, lower, diag, upper, rhs, Ais, Bis, Cis, Gis, RHS

In [ ]:
def _solve_main_grid_dirichlet(n, a=-1.0, b=1.0, ua=0.0, ub=0.0):
    x, h, lower, diag, upper, rhs, Ais, Bis, Cis, Gis, RHS = _assemble_main_tridiag_dirichlet(
        n, a=a, b=b, ua=ua, ub=ub
    )
    u_inner = _thomas_solve(lower, diag, upper, rhs)
    u = np.zeros(n + 1, dtype=float)
    u[0] = ua
    u[n] = ub
    u[1:n] = u_inner
    return x, u

In [ ]:
def _assemble_shifted_tridiag(n, a=-1.0, b=1.0):
    h = (b - a) / n
    xs = (a - 0.5 * h) + h * np.arange(n + 2)
    N = n + 2

    lower = np.zeros(N - 1, dtype=float)
    diag = np.zeros(N, dtype=float)
    upper = np.zeros(N - 1, dtype=float)
    rhs = np.zeros(N, dtype=float)

    diag[0] = 1.0
    upper[0] = 1.0
    rhs[0] = 0.0

    Ais = np.zeros(N, dtype=float)
    Bis = np.zeros(N, dtype=float)
    Cis = np.zeros(N, dtype=float)
    Gis = np.zeros(N, dtype=float)

    Bis[0] = diag[0]
    Cis[0] = upper[0]
    Gis[0] = rhs[0]

    for i in range(1, n + 1):
        xi = xs[i]
        pL = _p(xi - 0.5 * h)
        pR = _p(xi + 0.5 * h)

        Ai = -pL / (h * h)
        Ci = -pR / (h * h)
        Bi = (pL + pR) / (h * h) + _r(xi)
        Gi = _f(xi)

        lower[i - 1] = Ai
        diag[i] = Bi
        upper[i] = Ci
        rhs[i] = Gi

        Ais[i] = Ai
        Bis[i] = Bi
        Cis[i] = Ci
        Gis[i] = Gi

    lower[N - 2] = 1.0
    diag[N - 1] = 1.0
    rhs[N - 1] = 0.0

    Ais[N - 1] = lower[N - 2]
    Bis[N - 1] = diag[N - 1]
    Cis[N - 1] = 0.0
    Gis[N - 1] = rhs[N - 1]

    return xs, h, lower, diag, upper, rhs, Ais, Bis, Cis, Gis

In [ ]:
def _solve_shifted_grid_oh2(n, a=-1.0, b=1.0):
    xs, h, lower, diag, upper, rhs, Ais, Bis, Cis, Gis = _assemble_shifted_tridiag(n, a=a, b=b)
    y = _thomas_solve(lower, diag, upper, rhs)

    xm = a + h * np.arange(n + 1)
    um = 0.5 * (y[0:n + 1] + y[1:n + 2])
    return xm, um, xs, y

In [ ]:
def _ref_solution_thin_grid(n_ref=4000, a=-1.0, b=1.0):
    xr, ur = _solve_main_grid_dirichlet(n_ref, a=a, b=b, ua=0.0, ub=0.0)
    return xr, ur

In [ ]:
def _ref_u_on_grid_thin(x_eval, xr, ur):
    x_eval = np.asarray(x_eval, dtype=float)
    return np.interp(x_eval, xr, ur)

In [ ]:
def _solve_bvp_reference(a=-1.0, b=1.0, tol=1e-10, max_nodes=100000):
    try:
        from scipy.integrate import solve_bvp
    except Exception:
        raise _err("scipy не найден - выберите, пожалуйста, эталон через тонкую сетку :<")

    def fun(x, y):
        u = y[0]
        up = y[1]
        return np.vstack([
            up,
            (_r(x) * u - _f(x) - _pprime(x) * up) / _p(x)
        ])

    def bc(ya, yb):
        return np.array([ya[0], yb[0]])

    x_mesh = np.linspace(a, b, 50)
    y_guess = np.zeros((2, x_mesh.size), dtype=float)

    sol = solve_bvp(fun, bc, x_mesh, y_guess, tol=tol, max_nodes=max_nodes)
    if not sol.success:
        raise _err("solve_bvp не сошёлся: " + str(sol.message))
    return sol

In [ ]:
def _errs(u, uref, h):
    e = u - uref
    emax = float(np.max(np.abs(e)))
    el2 = float(np.sqrt(h * np.sum(e * e)))
    return emax, el2

In [ ]:
def _to_full_tridiag_matrix(lower, diag, upper):
    m = diag.size
    A = np.zeros((m, m), dtype=float)
    for i in range(m):
        A[i, i] = diag[i]
        if i - 1 >= 0:
            A[i, i - 1] = lower[i - 1]
        if i + 1 < m:
            A[i, i + 1] = upper[i]
    return A

In [ ]:
def _print_main_coeffs_for_report(n, a=-1.0, b=1.0, ua=0.0, ub=0.0):
    x, h, lower, diag, upper, rhs, Ais, Bis, Cis, Gis, RHS = _assemble_main_tridiag_dirichlet(
        n, a=a, b=b, ua=ua, ub=ub
    )
    display(Markdown(f"**коэффициенты для основной сетки (n={n})**\nh = `{h:.6g}`"))
    display(Markdown("мы решаем для неизвестных `u_1..u_{n-1}` уравнения вида\n`A_i*u_{i-1} + B_i*u_i + C_i*u_{i+1} = RHS_i`"))

    for i in range(1, n):
        k = i - 1
        display(Markdown(
            f"i=`{i}`, x_i=`{x[i]:.6g}`\n"
            f"A_i=`{Ais[k]:.12g}`\n"
            f"B_i=`{Bis[k]:.12g}`\n"
            f"C_i=`{Cis[k]:.12g}`\n"
            f"f(x_i)=`{Gis[k]:.12g}`\n"
            f"RHS_i=`{RHS[k]:.12g}`"
        ))

    T = _to_full_tridiag_matrix(lower, diag, upper)
    display(Markdown("**явная матрица системы (основная сетка)**"))
    display_latex(Mat(T), label=r"\mathbf{T}_{\mathrm{main}}")
    display(Markdown("**правая часть системы (основная сетка)**"))
    display_latex(Vec(rhs), label=r"\mathbf{g}_{\mathrm{main}}")

In [ ]:
def _print_shifted_coeffs_for_report(n, a=-1.0, b=1.0):
    xs, h, lower, diag, upper, rhs, Ais, Bis, Cis, Gis = _assemble_shifted_tridiag(n, a=a, b=b)
    N = n + 2
    display(Markdown(f"**коэффициенты для сдвинутой сетки (n={n})**\nh = `{h:.6g}`"))
    display(Markdown("неизвестные `y_0..y_{n+1}`\nграничные уравнения\n`y_0 + y_1 = 0`\n`y_n + y_{n+1} = 0`"))

    for i in range(0, N):
        Ai = (lower[i - 1] if 1 <= i <= N - 1 else 0.0)
        Bi = diag[i]
        Ci = (upper[i] if 0 <= i <= N - 2 else 0.0)
        display(Markdown(
            f"i=`{i}`, x_i(shift)=`{xs[i]:.6g}`\n"
            f"A_i=`{Ai:.12g}`\n"
            f"B_i=`{Bi:.12g}`\n"
            f"C_i=`{Ci:.12g}`\n"
            f"RHS_i=`{rhs[i]:.12g}`"
        ))

    T = _to_full_tridiag_matrix(lower, diag, upper)
    display(Markdown("**явная матрица системы (сдвинутая сетка)**"))
    display_latex(Mat(T), label=r"\mathbf{T}_{\mathrm{shift}}")
    display(Markdown("**правая часть системы (сдвинутая сетка)**"))
    display_latex(Vec(rhs), label=r"\mathbf{g}_{\mathrm{shift}}")

In [ ]:
def run_variant1(
    n_list=(10, 20),
    ref_kind="scipy",
    n_ref=4000,
    scipy_tol=1e-10,
    scipy_max_nodes=100000,
    a=-1.0,
    b=1.0,
    print_coeffs=False,
    coeffs_n=10
):
    ref_kind = _canon_ref_kind(ref_kind)

    display(Markdown(
        r"$-\left(\dfrac{1}{2+x}u'\right)'+\cos(x)\,u=1+x,\quad u(-1)=u(1)=0$"
    ))

    xr = None
    ur = None
    bvp_sol = None

    if ref_kind == "grid":
        xr, ur = _ref_solution_thin_grid(n_ref=n_ref, a=a, b=b)
        display(Markdown(f"**эталонное решение:** тонкая сетка n_ref=`{n_ref}`"))
        u_ref_on = lambda x: _ref_u_on_grid_thin(x, xr, ur)
    else:
        display(Markdown(f"**эталонное решение:** scipy.solve_bvp (tol=`{scipy_tol}`)"))
        bvp_sol = _solve_bvp_reference(a=a, b=b, tol=scipy_tol, max_nodes=scipy_max_nodes)
        u_ref_on = lambda x: bvp_sol.sol(np.asarray(x, dtype=float))[0]

    results = []
    curves = []  # для графиков (=^..^=)

    for n in n_list:
        h = (b - a) / n

        display(Markdown(f"**n = {n}**\nh = `{h:.6g}`"))

        if print_coeffs and int(n) == int(coeffs_n):
            _print_main_coeffs_for_report(n, a=a, b=b, ua=0.0, ub=0.0)
            _print_shifted_coeffs_for_report(n, a=a, b=b)

        x1, u1 = _solve_main_grid_dirichlet(n, a=a, b=b, ua=0.0, ub=0.0)
        uref_s = u_ref_on(x1)
        emax1, el21 = _errs(u1, uref_s, h)

        display(Markdown("**решение на основной сетке u(x_i)**"))
        label_main = r"u_{\mathrm{main}}^{(n=" + str(n) + r")}"
        display_latex(Vec(u1), label=label_main)

        x2, u2, xs, y = _solve_shifted_grid_oh2(n, a=a, b=b)
        emax2, el22 = _errs(u2, uref_s, h)

        display(Markdown("**решение со сдвинутой сетки, восстановленное на основной u(x_i)**"))
        label_shift = r"u_{\mathrm{shift}}^{(n=" + str(n) + r")}"
        display_latex(Vec(u2), label=label_shift)

        display(Markdown("**ошибки относительно эталона на узлах основной сетки**"))
        display(Markdown(
            f"основная сетка: max|e|=`{emax1:.6g}`\n"
            f"основная сетка: L2(e)=`{el21:.6g}`\n"
            f"сдвиг+усреднение: max|e|=`{emax2:.6g}`\n"
            f"сдвиг+усреднение: L2(e)=`{el22:.6g}`"
        ))

        results.append((n, emax1, el21, emax2, el22))
        curves.append({
            "n": n,
            "x": x1,
            "u_ref": uref_s,
            "u_main": u1,
            "u_shift": u2,
            "err_main": np.abs(u1 - uref_s),
            "err_shift": np.abs(u2 - uref_s),
        })

    display(Markdown("**сводка ошибок относительно эталона**"))
    for (n, emax1, el21, emax2, el22) in results:
        display(Markdown(
            f"n=`{n}`\n"
            f"O(h): max|e|=`{emax1:.6g}`, L2(e)=`{el21:.6g}`\n"
            f"O(h^2): max|e|=`{emax2:.6g}`, L2(e)=`{el22:.6g}`"
        ))

    x_plot = np.linspace(a, b, 800)
    u_plot_ref = u_ref_on(x_plot)

    display(Markdown("**график 1: сравнение эталонного и приближённых решений**"))
    plt.figure()
    plt.plot(x_plot, u_plot_ref, label="эталон")
    for c in curves:
        n = c["n"]
        plt.plot(c["x"], c["u_main"], marker="o", label=f"O(h), n={n}")
        plt.plot(c["x"], c["u_shift"], marker="o", label=f"O(h^2), n={n}")
    plt.xlabel("x")
    plt.ylabel("u(x)")
    plt.legend()
    plt.grid(True)
    plt.show()

    display(Markdown("**график 2: сравнение фактических погрешностей |e(x)| на узлах основной сетки**"))
    plt.figure()
    for c in curves:
        n = c["n"]
        plt.plot(c["x"], c["err_main"], marker="o", label=f"|e|, O(h), n={n}")
        plt.plot(c["x"], c["err_shift"], marker="o", label=f"|e|, O(h^2), n={n}")
    plt.xlabel("x")
    plt.ylabel("|e(x)|")
    plt.legend()
    plt.grid(True)
    plt.show()

    print("мяу, готово (=^..^=)")

In [ ]:
def _to_int(s, default):
    s = (s or "").strip()
    if s == "":
        return default
    return int(s)

In [ ]:
def _to_float(s, default):
    s = (s or "").strip()
    if s == "":
        return default
    return float(s)

In [ ]:
def main():
    display(Markdown(
        "мяу мяу,\n"
        "мы считаем на основной сетке (O(h)) и на сдвинутой с усреднением (O(h^2))\n"
        "и сравниваем с эталонным решением"
    ))

    n1 = _to_int(input("n1 (по умолчанию 10): "), 10)
    n2 = _to_int(input("n2 (по умолчанию 20): "), 20)

    ref_kind = _canon_ref_kind(input("эталон: `scipy` (solve_bvp) / `grid` (тонкая сетка) [по умолчанию scipy]: ").strip().lower() or "scipy")

    n_ref = 4000
    scipy_tol = 1e-10
    scipy_max_nodes = 100000

    if ref_kind == "grid":
        n_ref = _to_int(input("n_ref для тонкой сетки (по умолчанию 4000): "), 4000)
    else:
        try:
            import scipy
        except Exception:
            display(Markdown("**предупреждение:** scipy не найден, переключаюсь на `grid` (тонкая сетка)"))
            ref_kind = "grid"
            n_ref = _to_int(input("n_ref для тонкой сетки (по умолчанию 4000): "), 4000)

    if ref_kind == "scipy":
        scipy_tol = _to_float(input("tol для solve_bvp (по умолчанию 1e-10): "), 1e-10)
        scipy_max_nodes = _to_int(input("max_nodes для solve_bvp (по умолчанию 100000): "), 100000)

    ans = (input("печатать коэффициенты и явные матрицы системы? (y/n, по умолчанию n): ").strip().lower() or "n")
    print_coeffs = ans.startswith("y")

    coeffs_n = n1
    if print_coeffs:
        coeffs_n = _to_int(input(f"для какого n печатать коэффициенты? (по умолчанию {n1}): "), n1)

    run_variant1(
        n_list=(n1, n2),
        ref_kind=ref_kind,
        n_ref=n_ref,
        scipy_tol=scipy_tol,
        scipy_max_nodes=scipy_max_nodes,
        print_coeffs=print_coeffs,
        coeffs_n=coeffs_n
    )

In [ ]:
main()